In [1]:
path = "drive/My Drive/Colab Notebooks/466-proj2/"
f = open(path + 'readme.txt', 'r')
print(f.read())
f.close()

Digital Democracy Data Sets (for Project 2 groups)
The ordering for the tsv as well as what each represents (committee_utterance file)

1)vid                   - Video Id
2)fileid                - YouTube Id
3)cid                   - Committee Id
4)c_name                - Committee Name
5)c_house               - Committee House
6)hid                   - Hearing Id
7)position              - Order of bills discussed in the hearing/video order
8)pid                   - Person Id
9)diarization_id        - Id of a speaker given perfect diarization
10)last                 - Speaker last name
11)first                - Speaker first name
12)start                - Start time of utterance
13)end                  - End time of utterance
14)utterance_order      - Order of the utterance in a given hearing
15)text                 - Text of the utterance

DDDataSet_3.tsv

Use this data set for your baseline machine learning analysis of speaker identification. There are about 10,000 randomly selected 

In [2]:
FILE = "committee_utterances.tsv"

import pandas as pd

df = pd.read_csv(path + FILE, sep='\t')
df

,vid,fileid,cid,c_name,c_house,hid,position,pid,diarization_id,last,first,start,end,utterance_order,text
0,4527,8xcAFOvPC50,2,Agriculture,Assembly,648,1,95,1,Galgiani,Cathleen,6,24,1,"Go ahead and call the hearing to order, our sp..."
1,4527,8xcAFOvPC50,2,Agriculture,Assembly,648,1,95,1,Galgiani,Cathleen,24,41,2,But also to the communities and it's had other...
2,4527,8xcAFOvPC50,2,Agriculture,Assembly,648,1,95,1,Galgiani,Cathleen,41,56,3,limited water resources are all challenges tha...
3,4527,8xcAFOvPC50,2,Agriculture,Assembly,648,1,95,1,Galgiani,Cathleen,56,71,4,"And we've invited expert witnesses, who'll dis..."
4,4527,8xcAFOvPC50,2,Agriculture,Assembly,648,1,95,1,Galgiani,Cathleen,71,88,5,Drought is certainly a critical issue for Cali...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30858,2858,nMgNQIdoyHs,148,Public Health and Developmental Services,Senate,497,4,103,1,Hernandez,Ed,1485,1494,555,"Current vote count is 9-3, and that bill is ou..."
30859,2858,nMgNQIdoyHs,148,Public Health and Developmental Services,Senate,497,4,2998,4,Secretary,Committee,1494,1498,556,Anderson? Mitchell? (aye) Moorlach?
30860,2858,nMgNQIdoyHs,148,Public Health and Developmental Services,Senate,497,4,103,1,Hernandez,Ed,1504,1512,557,Current vote count is 9-2. That bill is out. I...
30861,2858,nMgNQIdoyHs,148,Public Health and Developmental Services,Senate,497,4,2998,4,Secretary,Committee,1512,1521,558,Morrell? (no) Anderson? Mitchell? (aye) 9-3.


In [91]:
#Select a random 25% (1/4) of the content
import random
from pprint import pprint

records = list(df.text)
number_selected = len(records) // 1000  #TODO CHANGE THIS VALUE TO 4
selected_records = random.sample(records, number_selected) #get random sample of number_selected records without replacement

utterances = [record for record in selected_records] #get all the utterance fields only (column #15) for clustering
print("total number of points:",len(utterances))

total number of points: 30


In [106]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords 
from nltk.corpus import wordnet 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

wordnet_lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english')) 
punctuation = "-!?;:\"\'.,"

def getPOS(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def getFeatures(text):
    features = {}
    tokens = nltk.word_tokenize(text.lower())
    tot_cnt = len(tokens)
    for word in tokens:        
        if word.isnumeric():
            label = '[NUMBER]'
            if label not in features:
                features[label] = 1/float(tot_cnt)    
            else:
                features[label] += 1/float(tot_cnt)    
        elif word not in stop_words and word not in punctuation:
            w = wordnet_lemmatizer.lemmatize(word, getPOS(word))
            if w not in features:
                features[w] = 1/float(tot_cnt)
            else:
                features[w] += 1/float(tot_cnt)
    return features


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [0]:
import numpy as np
import random, sys, copy
from collections import Counter

max_t = 3   # max itterations (ignores threshold) (-1 to unset)10
M = []
M.append([0 for i in range(0, k)])

def initCentroids(vec_data, k):
    centroidID = np.random.permutation(len(vec_data))[:k]
    print(centroidID)
    return [vec_data[cID] for cID in centroidID]

def calculateDistance(v_dict, u_dict):
    v = list(v_dict.values())
    u = list(u_dict.values())
    total = 0
    for i in range(0, len(v)):
        total += ((v[i] - u[i]) ** 2)
    return np.sqrt(total)


# takes a vector dictionary and a set of all the keys
# returns a new vector dictionary with all the keys
def reshapeVectDict(vect_dict, keys):
    result = copy.deepcopy(vect_dict)
    for key in keys:
        if key not in result:
            result[key] = 0.0
    return result

# For a given point, find the closest centroid from the list, return centroid's index
def closestCentroid(point, k, data):
    # Get all the keys (words) in the point and centroids
    # This will be used to transform them to the same dimension so we can do math
    all_keys = []
    all_keys.extend(point)        
    for cID in range(0,k):        
        all_keys.extend(data[cID].keys())
    all_keys = set(all_keys)

    closestCentroid = 0
    minDist = sys.maxsize
    point_reshape = reshapeVectDict(point, all_keys)

    # print()
    # print("point: ", point_reshape)
    for cID in range(0,k):
        centroid = data[cID]
        centroid_reshape = reshapeVectDict(centroid, all_keys)
        dist = calculateDistance(point_reshape, centroid_reshape)

        # print("dist: ", dist, "centroid: ", centroid)
        if (dist < minDist):
            minDist = dist          
            closestCentroid = cID

    # print("closestCentroid: ", closestCentroid, "minDist: ", minDist)
    return closestCentroid

def recalculateCentroid(cluster, data):
    new_centroid = {}
    for idx in cluster:
        point = data[idx]
        new_centroid = dict(Counter(new_centroid) + Counter(point))

    for key in new_centroid:
        new_centroid[key] /= len(cluster)
    # print("new_centroid: ", new_centroid)
    return new_centroid

def k_means(vec_data, k, e):
    t = 0
    M[t] = initCentroids(vec_data, k)

    if max_t >= 0:        
        while (t < max_t):
            print("iteration: ", t)
            t += 1
            M.append([0 for i in range(0, k)])    # M needs t rows
            C = []
            for i in range(0, k):
                C.append([])
            # Centroid assignment
            for point in vec_data:
                clusterID = closestCentroid(point, k, M[t-1])                
                C[clusterID].append(vec_data.index(point))    # keep track of the point's as indices
            print(len(C), C)
            
            # Centroid update
            for i in range(0, k):
                new_centroid = recalculateCentroid(C[i], vec_data)
                M[t][i] = new_centroid

In [125]:
k = 3
centroids = initCentroids(vec_data, k)
centroids

[22 11  8]


[{'21st': 0.034482758620689655,
  'ago': 0.034482758620689655,
  'century': 0.034482758620689655,
  'decade': 0.034482758620689655,
  'design': 0.034482758620689655,
  'fair': 0.034482758620689655,
  'many': 0.034482758620689655,
  'obviously': 0.034482758620689655,
  'program': 0.034482758620689655,
  'question': 0.034482758620689655,
  'suitable': 0.034482758620689655,
  'whether': 0.034482758620689655,
  'year': 0.034482758620689655},
 {"'s": 0.05555555555555555,
  'bank': 0.05555555555555555,
  'big': 0.027777777777777776,
  'concern': 0.027777777777777776,
  'cost': 0.027777777777777776,
  'get': 0.027777777777777776,
  'go': 0.027777777777777776,
  'know': 0.027777777777777776,
  'loan': 0.027777777777777776,
  'money': 0.027777777777777776,
  "n't": 0.027777777777777776,
  'true': 0.027777777777777776},
 {"'s": 0.045454545454545456,
  '987.': 0.045454545454545456,
  '[NUMBER]': 0.09090909090909091,
  'ab': 0.045454545454545456,
  'clerk': 0.045454545454545456,
  'file': 0.090909

In [132]:
recalculateCentroid([22, 11, 8], vec_data)

new_centroid:  {'whether': 0.011494252873563218, 'program': 0.011494252873563218, 'obviously': 0.011494252873563218, 'design': 0.011494252873563218, 'many': 0.011494252873563218, 'year': 0.011494252873563218, 'decade': 0.011494252873563218, 'ago': 0.011494252873563218, 'suitable': 0.011494252873563218, '21st': 0.011494252873563218, 'century': 0.011494252873563218, 'fair': 0.011494252873563218, 'question': 0.011494252873563218, 'cost': 0.009259259259259259, 'true': 0.009259259259259259, "'s": 0.03367003367003367, 'get': 0.009259259259259259, 'go': 0.009259259259259259, 'bank': 0.018518518518518517, 'loan': 0.009259259259259259, 'money': 0.009259259259259259, "n't": 0.009259259259259259, 'know': 0.009259259259259259, 'big': 0.009259259259259259, 'concern': 0.009259259259259259, 'file': 0.030303030303030304, 'item': 0.030303030303030304, '[NUMBER]': 0.030303030303030304, 'pas': 0.015151515151515152, 'retain': 0.015151515151515152, 'move': 0.015151515151515152, 'ab': 0.015151515151515152, 

{"'s": 0.03367003367003367,
 '21st': 0.011494252873563218,
 '987.': 0.015151515151515152,
 '[NUMBER]': 0.030303030303030304,
 'ab': 0.015151515151515152,
 'ago': 0.011494252873563218,
 'bank': 0.018518518518518517,
 'big': 0.009259259259259259,
 'century': 0.011494252873563218,
 'clerk': 0.015151515151515152,
 'concern': 0.009259259259259259,
 'cost': 0.009259259259259259,
 'decade': 0.011494252873563218,
 'design': 0.011494252873563218,
 'fair': 0.011494252873563218,
 'file': 0.030303030303030304,
 'get': 0.009259259259259259,
 'go': 0.009259259259259259,
 'item': 0.030303030303030304,
 'know': 0.009259259259259259,
 'loan': 0.009259259259259259,
 'many': 0.011494252873563218,
 'money': 0.009259259259259259,
 'move': 0.015151515151515152,
 "n't": 0.009259259259259259,
 'obviously': 0.011494252873563218,
 'pas': 0.015151515151515152,
 'program': 0.011494252873563218,
 'question': 0.011494252873563218,
 'read': 0.015151515151515152,
 'retain': 0.015151515151515152,
 'suitable': 0.011494

In [128]:
closestCentroid(vec_data[0], k, centroids)


point:  {'thank': 0.06666666666666667, 'witness': 0.06666666666666667, 'opposition': 0.06666666666666667, 'measure': 0.06666666666666667, 'say': 0.06666666666666667, 'none': 0.06666666666666667, 'finance': 0.06666666666666667, 'read': 0.0, 'go': 0.0, 'century': 0.0, 'ab': 0.0, 'bank': 0.0, '21st': 0.0, 'money': 0.0, 'obviously': 0.0, 'year': 0.0, 'decade': 0.0, 'question': 0.0, 'move': 0.0, 'many': 0.0, "n't": 0.0, 'cost': 0.0, 'ago': 0.0, 'item': 0.0, 'concern': 0.0, '[NUMBER]': 0.0, 'get': 0.0, 'retain': 0.0, 'file': 0.0, 'pas': 0.0, '987.': 0.0, 'loan': 0.0, "'s": 0.0, 'fair': 0.0, 'program': 0.0, 'whether': 0.0, 'true': 0.0, 'suitable': 0.0, 'big': 0.0, 'know': 0.0, 'design': 0.0, 'clerk': 0.0}
dist:  0.11993744791490188 centroid:  {'whether': 0.034482758620689655, 'program': 0.034482758620689655, 'obviously': 0.034482758620689655, 'design': 0.034482758620689655, 'many': 0.034482758620689655, 'year': 0.034482758620689655, 'decade': 0.034482758620689655, 'ago': 0.034482758620689655

1

In [0]:
data = selected_records
vec_data = [getFeatures(text) for text in data]
print(len(vec_data))
vec_data

In [140]:
import time

startTime = time.time()
endTime = time.time()

k_means(vec_data, 3, 1)

print(round(endTime - startTime,3),'seconds')

[ 5 23 13]
iteration:  0
3 [[3, 4, 5, 10, 11, 14, 15, 17, 22, 24, 28], [12, 16, 18, 19, 20, 23, 26, 29], [0, 1, 2, 6, 7, 8, 9, 13, 21, 25, 27]]
iteration:  1
3 [[], [4, 5, 10, 12, 13, 16, 18, 19, 20, 23, 26, 29], [0, 1, 2, 3, 6, 7, 8, 9, 11, 14, 15, 17, 21, 22, 24, 25, 27, 28]]
iteration:  2
3 [[], [3, 4, 5, 12, 15, 16, 18, 19, 20, 23, 24, 26, 28, 29], [0, 1, 2, 6, 7, 8, 9, 10, 11, 13, 14, 17, 21, 22, 25, 27]]
0.0 seconds


In [12]:
# M holds the centroid IDs
M

[[67, 44], [0, 0], [0, 0]]

In [49]:
print(len(vec_data[0]), vec_data[0])
print(len(vec_data[1]), vec_data[1])
new_d = recalculateCentroid([0,1], vec_data)
print(len(new_d), new_d)

30 {'thank': 0.016129032258064516, 'much': 0.016129032258064516, 'would': 0.016129032258064516, 'like': 0.016129032258064516, 'say': 0.016129032258064516, 'recognize': 0.016129032258064516, 'critical': 0.016129032258064516, 'initial': 0.016129032258064516, 'step': 0.016129032258064516, 'take': 0.016129032258064516, 'tackle': 0.016129032258064516, 'issue': 0.016129032258064516, 'appreciate': 0.016129032258064516, 'finalize': 0.016129032258064516, 'data': 0.04838709677419355, 'share': 0.016129032258064516, 'agreement': 0.016129032258064516, 'systematically': 0.016129032258064516, 'review': 0.016129032258064516, 'identify': 0.016129032258064516, 'course': 0.016129032258064516, 'action': 0.016129032258064516, 'base': 0.016129032258064516, 'really': 0.016129032258064516, 'help': 0.016129032258064516, 'ensure': 0.016129032258064516, 'foster': 0.016129032258064516, 'kid': 0.016129032258064516, 'inappropriately': 0.016129032258064516, 'medicate': 0.016129032258064516}
13 {'jennifer': 0.0357142

In [40]:
dict1 = {'a':0, 'b':0}
dict2 = dict1
dict2['a'] += 1

print("dict1: ", dict1)
print("dict2: ", dict2)

dict1:  {'a': 1, 'b': 0}
dict2:  {'a': 1, 'b': 0}
